In [1]:
import torch
import math
import random

In [2]:
class Value():
    def __init__(self,data,children = ()):
        self.data = data
        self.grad = 0.0 # derivate of L in backward propagation
        self._backward = lambda : None
        self.prev = children
    
    def __repr__(self):
        return f'Value:data({self.data})'
    
    
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other))
        
        def _backward():
            self.grad += 1.0 * out.grad  
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other))
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out


    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t,(self,))

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other,(self,))
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out
    
    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __radd__(self, other): # other + self
        return self + other
    

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
          if v not in visited:
            visited.add(v)
            for child in v.prev:
              build_topo(child)
            topo.append(v)
        build_topo(self)  

        self.grad = 1.0

        for node in reversed(topo):
            node._backward()


In [7]:
#test Example 1

x1 = Value(2.0)
x2 = Value(0.0)

w1 = Value(-3.0)
w2 = Value(1.0)

b = Value(6.7)

x1w1 = x1*w1

x2w2 = x2*w2

x1w1x2w2 = x1w1 + x2w2 

n = x1w1x2w2 + b 

o = n.tanh()
o

Value:data(0.6043677771171636)

In [8]:
o.backward()

In [9]:
print(o.grad, n.grad,x1w1x2w2.grad, x1w1.grad, x2w2.grad,x2.grad, x1.grad,w2.grad,w1.grad )

1.0 0.6347395899824584 0.6347395899824584 0.6347395899824584 0.6347395899824584 0.6347395899824584 -1.9042187699473752 0.0 1.2694791799649168


Building Neural Network

In [8]:
class Neuron():
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for i in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self,x):
        # w*x + b
        h = sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
        o = h.tanh()
        return o
    def parameters(self):
        return self.w + [self.b]
    

class Layer():
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for i in range(nout)]

    def __call__(self,x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        out = []
        for i in self.neurons:
            out.extend(i.parameters())
        return out
    

class MLP():
    def __init__(self,nin,nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]

    def __call__(self,x):
        for i in self.layers:
            layer = i(x)
        return layer
    
    def parameters(self):
        out = []
        for i in self.layers:
            out.extend(i.parameters())
        return out

In [12]:
n = MLP(3,[4,4,1])
x = [1.0,2.0]
n(x)

Value:data(0.3387541878121056)

In [17]:
n.parameters()[0].data

-0.23180699710307784